In [16]:
import pandas as pd
import numpy as np
from scipy import linalg
'''
规定样本矩阵尺寸：n*p  n:样本数    p:指标数

输入：
file_path：样本矩阵路径  在创建对象时直接输入
threshold：累计贡献率阈值

'''


class PCA:
    def __init__(self, file_path,threshold):
        """
        初始化类，读取样本矩阵并转换为 NumPy 数组。
        file_path: 表格文件路径

        参数解释：
        self.file_path：表格路径
        self.df：表格的dataframe类型数据
        self.matrix：表格的array类型
        self.threshold:累计贡献率阈值
        """
        
        self.file_path = file_path
        #读取excel文件
        self.df = pd.read_excel(file_path)
        
        #读取csv文件
        #self.df = pd.read_csv(file_path)
        self.matrix = self.df.to_numpy()
        self.threshold=threshold

    #标准化
    def standardize_function(self):
        """
        标准化数据（Z-score 标准化）
        """

        #np.mean(self.matrix, axis=0)：axis=1表示对每一列取均值，也就是每个指标的均值（平均身高、平均体重等）
        #np.std(self.matrix, ddof=1, axis=0)：ddof=1表示使用 无偏估计 的标准差   axis=0表示对每一列求标准差
        #self.matrix_biaozhunhua：标准化后的矩阵
        self.matrix_biaozhunhua = (self.matrix - np.mean(self.matrix, axis=0)) / np.std(self.matrix, ddof=1, axis=0)


    #计算协方差矩阵
    def xiefangcha_function(self):
        '''
        self.matrix_biaozhunhua.T:p*n矩阵，p为指标样本，n为样本数
        self.matrix_covariance：p*p矩阵
        
        协方差矩阵中：
        1.对角线元素为每个指标的方差
        2.除对角线元素外为协方差
        '''
        #将协方差矩阵传回self
        self.matrix_covariance = np.cov(self.matrix_biaozhunhua.T)
        

    #
    def calculate_eigenvalues_and_eigenvectors_function(self):
        """
        self.tezhengzhi:特征值，为1*p的矩阵
        self.vectors_tezheng:特征向量，为p*p的矩阵

        linalg.eigh返回升序的特征值和对应向量
        通过反转顺序将特征值按照降序排列
        """
        
        self.tezhengzhi, self.vectors_tezheng = linalg.eigh(self.matrix_covariance)
        # 将特征值和特征向量按降序排列
        self.tezhengzhi = self.tezhengzhi[::-1]
        self.vectors_tezheng = self.vectors_tezheng[:, ::-1]


    #计算主成分贡献率和累积贡献率。
    def gongxianlv_function(self):
        """
        self.gongxianlv:贡献率   1*p矩阵
        self.gongxianlv_leiji:累计贡献率  1*p矩阵

        方法解释：
        np.cumsum：计算（到某一个元素的）累计和
        """
        
        #计算贡献率
        self.gongxianlv = self.tezhengzhi / sum(self.tezhengzhi)

        #计算累计贡献率
        self.gongxianlv_leiji = np.cumsum(self.gongxianlv)


    #根据累积贡献率降维
    def jiangwei_function(self):
        """
        threshold：阈值，表示保留主成分的累计贡献率
        n_components：实际的主成分个数
        self.vectors_tezheng_zhuchengfen:主成分对应特征向量,每一列（p*1）为一个特征向量     p*n_components矩阵
        self.matrix_biaozhunhua：标准化矩阵   n*p矩阵
        self.matrix_jiangwei：n*n_components矩阵，即降维后的样本矩阵
        
        return: 降维后的数据
        """
        # 找到累积贡献率大于阈值的第一个主成分
        n_components = np.argmax(self.gongxianlv_leiji >= self.threshold) + 1  #"+1"是因为数组索引少1
        self.n_components=n_components #将主成分个数传回self
        
        # 选择前 n_components 个特征向量
        self.matrix_jiangwei = self.matrix_biaozhunhua @ self.vectors_tezheng[:, :n_components]

        #返回主成分对应的特征向量
        self.vectors_tezheng_zhuchengfen=self.vectors_tezheng[:, :n_components]

    #运行主成分分析
    def run(self):
        """
        运行主成分分析并返回降维后的结果。
        threshold: 累积贡献率阈值，默认值为 0.95
        return: 降维后的数据(样本矩阵)
        """
        self.standardize_function()
        self.xiefangcha_function()
        self.calculate_eigenvalues_and_eigenvectors_function()
        self.gongxianlv_function()
        return self.jiangwei_function()

    def print_results(self):
        """
        打印特征值、贡献率、累积贡献率和特征向量。
        """
        print('特征值为：')
        print(self.tezhengzhi)
        print('贡献率为：')
        print(self.gongxianlv)
        print('累计贡献率为：')
        print(self.gongxianlv_leiji)
        print('与特征值对应的特征向量矩阵为：')
        print(self.vectors_tezheng)
        print('主成分的个数为',self.n_components)
        print('主成分对应的特征向量为：')
        print(self.vectors_tezheng_zhuchengfen)
        

In [18]:
# 使用示例
if __name__ == "__main__":
    # 实例化类并运行分析
    pca = PCA('棉花产量论文作业的数据.xlsx',0.95)
    matrix_jiangwei = pca.run()
    
    # 打印降维后的结果
    print('降维后的矩阵为：')
    print(matrix_jiangwei)
    
    # 打印特征值、贡献率等结果
    pca.print_results()

降维后的矩阵为：
None
特征值为：
[5.60690976e+00 1.02248804e+00 2.18364846e-01 1.08328097e-01
 2.25384905e-02 1.69596595e-02 4.41110721e-03]
贡献率为：
[8.00987109e-01 1.46069719e-01 3.11949780e-02 1.54754424e-02
 3.21978435e-03 2.42280851e-03 6.30158173e-04]
累计贡献率为：
[0.80098711 0.94705683 0.97825181 0.99372725 0.99694703 0.99936984
 1.        ]
与特征值对应的特征向量矩阵为：
[[-0.41050057 -0.09508243  0.13962939 -0.57447388 -0.3131281   0.40520068
   0.45901548]
 [-0.34030144 -0.45116007 -0.79397455  0.21135383 -0.04589949  0.02899523
   0.05134206]
 [-0.41767355 -0.0729833   0.21472653  0.08583273 -0.12786291  0.38671635
  -0.77514453]
 [-0.40591796  0.21068021  0.16477147  0.33198307 -0.59525529 -0.53526722
   0.11243757]
 [-0.19710187  0.85487078 -0.39163483  0.0249426   0.12636961  0.24479604
   0.021271  ]
 [-0.40770146 -0.0827776   0.35064326  0.49314419  0.54985133  0.15585926
   0.36627435]
 [-0.41399758  0.00671031 -0.02285609 -0.51372753  0.45919547 -0.56097588
  -0.19638784]]
主成分的个数为 3
主成分对应的特征向量为：
[[-0.41